In [ ]:
import os
import json
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle

%matplotlib inline

def flatten_dict(d):
    out = {}
    for key, val in d.items():
        if isinstance(val, dict):
            val = [val]
        if isinstance(val, list):
            for subdict in val:
                deeper = flatten_dict(subdict).items()
                out.update({key + '.' + key2: val2 for key2, val2 in deeper})
        else:
            out[key] = val
    return out

prefix = "../vardim_icml_realizable_2022-05-09/"
# prefix = "../vardim_icml_real_nohls_2022-05-06/"
# prefix = "../vardim_icml_nonrealizable_2022-05-06/"
prefix = "../mixing_2022-05-16/"
folders = [
    f"{prefix}leader",
    # f"{prefix}srlegreedy_mineig",
    f"{prefix}srlegreedy_mineig_norm",
    # f"{prefix}srlegreedy_avg_quad",
    # f"{prefix}srlegreedy_avg_quad_norm",
    # f"{prefix}srllinucb_mineig",
    f"{prefix}srllinucb_mineig_norm",
    # f"{prefix}srllinucb_avg_quad",
    # f"{prefix}srllinucb_avg_quad_norm",
    # f"{prefix}egreedyglrt_0",
    # f"{prefix}egreedyglrt_1",
    # f"{prefix}egreedyglrt_2",
    # f"{prefix}egreedyglrt_3",
    # f"{prefix}egreedyglrt_4",
    # f"{prefix}egreedyglrt_5",
    # f"{prefix}linucb_0",
    # f"{prefix}linucb_1",
    # f"{prefix}linucb_2",
    # f"{prefix}linucb_3",
    # f"{prefix}linucb_4",
    # f"{prefix}linucb_5"
]
df = None
result_keys = []
fcnt = 0
for rootdir in folders:
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            if os.path.exists(os.path.join(d, "result.pkl")):
                fcnt+=1
                with open(os.path.join(d, "config.json"), "r") as jf:
                    config = json.load(jf)
                    config = flatten_dict(config)

                with open(os.path.join(d, "result.pkl"), "rb") as jf:
                    results = pickle.load(jf)
                # rank = config['newrank']
                # algo = config['algo']
                # layers = config['layers']
                # name = f"{algo}_{rank}_{layers}"
                for k in results.keys():
                    config[k] = [results[k]]
                
                result_keys = np.unique(result_keys + list(results.keys())).tolist()
                # else:
                #     assert np.all([el in result_keys for el in results.keys()])
                tdf = pd.DataFrame.from_dict(config)
                if df is None:
                    df = tdf
                else:
                    df = pd.concat((df,tdf))
            else:
                print(f"{d} not finished")



assert df.shape[0] == fcnt
print(f"result_keys:\n {result_keys}")
print(f"df.columns.to_list():\n {df.columns.to_list()}")

In [ ]:
# df

In [ ]:
keys = df.columns[~df.columns.isin(result_keys+['seed'])]
m2 = (df[keys] == df[keys].iloc[0]).all()
gk = keys[~m2].values.tolist()
print("keys for grouping: ", gk)
if len(gk) == 0:
    gk = "algo"
    print("new keys for grouping: ", gk)
if "algo" not in gk:
    gkn = gk
    gk = ["algo"]
    gk.extend(gkn)
print("keys for grouping: ", gk)
grouped = df.groupby(gk)

max_t = None
plt.figure(figsize=(15,10))
data = {}
for name, group in grouped:
    print(name)
    regrets = np.stack(group['regret'].values, axis=0)
    reps = regrets.shape[0]
    mean_regret = regrets.mean(axis=0)[:max_t]
    std_regret = regrets.std(axis=0)[:max_t] / np.sqrt(reps)
    data[name] = {
        "mean_regret": mean_regret,
        "ci": std_regret
    }
    if "leader" in name:
        plt.plot(mean_regret, "--", label=f"{name}")
    elif name[0].startswith("leaderselect_mineig"):
        plt.plot(mean_regret, ":", label=f"{name}")
    else:
        plt.plot(mean_regret, label=f"{name}")
    plt.fill_between(np.arange(mean_regret.shape[0]), mean_regret -2*std_regret, mean_regret+2*std_regret, alpha=0.1)

plt.legend(loc='upper left')
plt.title(f"{gk}")


pickle.dump(data, open( f"{prefix}results.p", "wb" ) )

In [ ]:
algs = [
# ('egreedyglrt', True, 'sqrt', 0, 'None') ,
# ('leader', False, 'cbrt', 0, 'None') ,
# ('linucb', False, 'sqrt', 0, 'None') ,
# ('linucb', False, 'sqrt', 1, 'None') ,
# ('linucb', False, 'sqrt', 2, 'None') ,
# ('linucb', False, 'sqrt', 3, 'None') ,
# ('linucb', False, 'sqrt', 4, 'None') ,
('srlegreedy_avg_quad_norm', True, 'cbrt', 0, 3) ,
('srlegreedy_mineig_norm', True, 'cbrt', 0, 1) ,
# ('srllinucb_avg_quad_norm', True, 'cbrt', 0, 3) ,
# ('srllinucb_mineig_norm', True, 'cbrt', 0, 1) ,
]

max_t = None
plt.figure(figsize=(30,20))
for name in algs:
    group = grouped.get_group(name)
    regrets = np.stack(group['regret'].values, axis=0)
    reps = regrets.shape[0]
    mean_regret = regrets.mean(axis=0)[:max_t]
    std_regret = regrets.std(axis=0)[:max_t] / np.sqrt(reps)
    if "leader" in name:
        plt.plot(mean_regret, "--", label=f"{name}")
    elif name[0].startswith("leaderselect_"):
        plt.plot(mean_regret, ":", label=f"{name}")
    else:
        plt.plot(mean_regret, label=f"{name}")
    plt.fill_between(np.arange(mean_regret.shape[0]), mean_regret -2*std_regret, mean_regret+2*std_regret, alpha=0.1)

plt.ylim(bottom=0)
plt.legend(loc='upper left')